In [ ]:
%matplotlib inline

import pandas as pd
from matplotlib import pyplot as plt

# Enunciado

2)  (\*\*\*\*) (15 pts) El GCPD (Gotham City Police Dep
t) recolecta la información de casos policiales que
 acontecen en 
Ciudad Gótica. Esta información se encuentra guarda
da en un dataframe con el siguiente formato: 
(fecha, id_caso, 
descripcion, estado_caso, categoria, latitud, longi
tud).  
Los  posibles estados  que puede tener  un caso son 1:
 caso abierto, 2: caso  resuelto, 3: cerrado sin res
olución.  Las 
fechas se encuentran en el formato YYYY-MM-DD. 
Por otro lado el comisionado Gordon guarda un regis
tro detallado sobre en cuáles casos fue activada la
 batiseñal para 
pedir ayuda del vigilante, Batman. Esta información
 se encuentra en un Dataframe con el siguiente formato
 (id_caso, respuesta)
siendo campo respuesta si la señal tuvo una respu
esta positiva (1) o negativa (0) de parte de él 
El  sector  encargado  de  las  estadísticas  oficiales  d
el  GCPD  quiere  con  esta  información
analizar  las  siguientes situaciones: 
- Tasa de resolución de casos de la fuerza policial
 por categoría de caso (considerando aquellos casos
 en los que no 
participó Batman). 
- Tasa de resolución de casos con la ayuda de Batma
n (considerando que aquellos casos en los que fue l
lamado con la batiseñal, participó en la resolución).
- Indicar el mes del año pasado en el que Batman tuvo
mayor participación en la investigación de casos.

# A tener en cuenta previamente

- Nos piden tres cosas diferentes. Si las tomamos como que las hacemos todas
  es más eficiente hacer primero el join y después ir filtrando por lo que nos piden. Si los tomamos
  por separado es más eficiente primero filtrar y después hacer el join.

---
# Creación de los datos


In [ ]:
d = {'fecha': ['2018-04-23', '2018-05-20', '2018-05-20'], 'id_caso': [1, 2, 3], 'descripcion': ['a', 'b', 'c'], 'estado_caso': [2, 2, 2], 'categoria': [1, 2, 3], 'latitud': [1, 2, 3], 'longitud': [1, 2, 3]}
casos = pd.DataFrame(d)
casos

In [ ]:
d = {'id_caso': [1, 2], 'respuesta': [1, 0]}
casos_batman = pd.DataFrame(d)
casos_batman

---
# Tasa de resolución de casos por categoría de caso

In [ ]:
casos['resuelto'] = casos['estado_caso'] == 2
casos

In [ ]:
casos.groupby('categoria').mean()

In [ ]:
resueltos = casos.loc[casos.estado_caso == 2]
resueltos

In [ ]:
tasa_resueltos_por_categoria = resueltos['categoria'].value_counts(normalize = True)
tasa_resueltos_por_categoria

**La tasa la tenemos que calcular con respecto al total de casos**
Como lo estamos haciendo acá, el resultado que obtenemos es el porcentaje de resueltos por categoría con
respecto al total de casos resueltos!

---
# Tasa de resolución de casos con ayuda de Batman

### Hacemos join con los casos

In [ ]:
casos_join = pd.DataFrame()
casos_join = casos.join(casos_batman.set_index('id_caso'), on = 'id_caso')
casos_join

- Tener en cuenta que el **join te settea a `how='left'`**, y como en el left tenemos el df de casos que tiene
 todos los casos totales, no perdemos información.
- Ver que nos queda un NaN, ese es porque es un caso que no estaba en el df de casos_batman, entonces no se tenía la información de respuesta.
- Si agarrabamos con `'right'` (`casos_batman`) perdemos todos aquellos casos que no están es este df
 Osea, que perdemos todos los casos en donde no se llamo a la batiseñal.
- Se puede hacer con `merge` también, pero me es más lindo así. :)

### Calculo tasa de resolución de casos con ayuda de batman

In [ ]:
tasas = casos_join.respuesta.value_counts(normalize = True)
tasas

In [ ]:
print('La tasa de resolución de casos con ayuda de batman es: ' + str(tasas[1]))

---
# Mes con más ayuda de Batman

In [ ]:
# Paso a timestamp fecha
casos_join['fecha'] = pd.to_datetime(casos_join.fecha)

In [ ]:
# Obtengo el mes
casos_join['mes'] = casos_join.fecha.dt.month
casos_join

In [ ]:
casos_join.loc[casos_join.respuesta == 1].mes.value_counts()

Se puede hacer con groupby?